In [ ]:
import sys
sys.path.insert(0, "/home/katharina/vame_approach/VAME")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from vame.analysis.kinutils import KinVideo, create_grid_video
import os
from datetime import datetime
from vame.util.auxiliary import read_config
import logging
import re
from pathlib import Path
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(asctime)s: %(message)s')


%reload_ext autoreload
%autoreload 2

## Load latent vectors

In [ ]:
PROJECT_PATH = "/home/katharina/vame_approach/themis_tail_belly_align"
# multiply with window length to find
min_dist_nn_factor = 2
dist_between_neighbors_factor = 0.5
selected_idx = 100
# load the CONFIG FILE from the last trained model
trained_models = [
    (datetime.strptime(element, "%m-%d-%Y-%H-%M"), element)
    for element in os.listdir(os.path.join(PROJECT_PATH, "model"))
]
# sort by time step
trained_models.sort(key=lambda x: x[0])
latest_model = trained_models[-1][-1]

config_file = os.path.join(PROJECT_PATH, "model", latest_model, "config.yaml")
config = read_config(config_file)

# select landmark file
landmark_file = config["video_sets"][0]
data_path = os.path.join(
        PROJECT_PATH,
        "results",
        latest_model,
        landmark_file,
        config["model_name"],
        "kmeans-" + str(config["n_init_kmeans"]),
    )
latent_vectors = np.load(
        os.path.join(data_path, "latent_vector_" + landmark_file + ".npy")
    )

In [ ]:
# extract landmark data and names
landmarks_orig = pd.read_csv(
        os.path.join(PROJECT_PATH, "landmarks", landmark_file + ".csv"), header=[0, 1],
    )
column_names = landmarks_orig.columns
landmark_names = [col_name[0] for col_name in column_names if col_name[-1] == "x"]

landmark_data_file = os.path.join(
        PROJECT_PATH, "data", landmark_file, landmark_file + "-PE-seq.npy"
    )
landmark_data_aligned = np.load(landmark_data_file).T
# reshape to (N_samples, N_landmarks, 2)
landmark_data_aligned = landmark_data_aligned.reshape(landmark_data_aligned.shape[0], -1, 2)

## Sample anchor latent embedding and visualize together with its nearest neighbors

In [ ]:
window_start = max(0, selected_idx - int(config["time_window"] * min_dist_nn_factor))
window_end = min(len(latent_vectors), selected_idx + int(config["time_window"] * min_dist_nn_factor))

selected_data = latent_vectors[selected_idx, :]
time_points = np.arange(0, latent_vectors.shape[0])
time_points = np.concatenate([time_points[0:window_start], time_points[window_end:-1]])
latent_vectors = np.concatenate([latent_vectors[0:window_start], latent_vectors[window_end:-1]])
    
neighbors = NearestNeighbors(n_neighbors=8).fit(latent_vectors)
dist, neighbor_idx = neighbors.kneighbors(selected_data.reshape(1,-1))
print(neighbor_idx.shape)
# all dist to selected latent vector
dist = np.sqrt(np.sum((latent_vectors - selected_data.reshape(1,-1))**2, axis=1))
dist_temporal = np.sqrt((time_points - selected_idx)**2)
# calc min spanning rectangle - if the rat is moving fast - 
# save aligned and cropped mouse video instead?


In [ ]:
# get corresponding video
video_df = pd.read_csv(os.path.join(PROJECT_PATH, "video_info.csv"))
video_id = int(re.findall(r"\d+", landmark_file)[0])
video_file = os.path.join(
            *video_df[video_df["video_id"] == video_id][
                ["vid_folder", "vid_file"]
            ].values[0]
        )
subject, date, camera_pos, video_name = Path(video_file).parts[-4:]
video = KinVideo(video_file, view=camera_pos)
video.probevid()

In [ ]:
# Sample
time_ids = [selected_idx, *time_points[neighbor_idx.reshape(-1)]]
# change from frames to seconds by dividing with fps
video_clip_data = [(video_file, t_id/ video.getfps(), (0,0,video.width,video.height)) for t_id in time_ids]
print(video_clip_data)
grid_video_name = create_grid_video(video_clip_data,3,speed=0.5) # duration is in seconds!!

In [ ]:
from IPython import display
display.Video(grid_video_name, embed=True,html_attributes="loop autoplay", width=900,height=900)


## T.SNE of latent space

In [9]:
from sklearn.manifold import TSNE

tsne = TSNE(perplexity=30).fit_transform(latent_vectors)

KeyboardInterrupt: 

In [ ]:
%matplotlib widget
plt.plot (tsne [:, 0], tsne [:, 1], 'r.')

In [ ]:
## Mark time windows with little and strong movement (lower and upper N% percentiles)

In [ ]:
# visualize time points with high motility
# idea: calc sum of differences in the tailbase - because when the rat is getting 
# up the tailbase will also remain constant
# use the non aligned data 
tailbase_pos = landmarks_orig[[("tailbase","x"), ("tailbase","y")]][1:]
tailbase_diff = tailbase_pos.diff(periods=1)
tailbase_shift = (tailbase_diff[("tailbase", "x")]**2 + tailbase_diff[("tailbase", "y")]**2)**0.5

In [ ]:
tailbase_shift.head(6)

In [ ]:
# calc per window the sum of position changes
total_vel = [np.sum(tailbase_shift[i:i+config["time_window"]]) for i in range(0,len(tailbase_shift)-config["time_window"])]
total_vel = np.array(total_vel)

In [ ]:
%matplotlib widget
plt.plot(np.arange(0,len(total_vel)),total_vel)
plt.xlabel("time point")
plt.ylabel("Sum of pos. differences")

In [ ]:
# select slowest 20% as resting; fastest 20% as walking
slow_percentile = np.percentile(total_vel, 5)
fast_percentile = np.percentile(total_vel, 95)
print(f"Lower percentile {slow_percentile}; upper percentile {fast_percentile}")

In [ ]:
%matplotlib widget
slow_time_points = total_vel < slow_percentile
fast_time_points = total_vel > fast_percentile

# plot s
plt.plot (tsne[:, 0], tsne[:, 1], 'ro', label='t.SNE')
plt.plot (tsne [fast_time_points, 0], tsne [fast_time_points, 1], 'bo', label='Fast')
plt.plot (tsne [slow_time_points, 0], tsne [slow_time_points, 1], 'yo', label='Slow')
plt.legend ()


In [ ]:
video.probevid()

In [ ]:
video.getnframes () / video.getfps () / 2.56

In [ ]:
# generate a short video clip

secbeg = 250
secend = 271
seclen = secend - secbeg


clip = video.clip (f'{secbeg}', seclen, scale=.3, speed=1., fps='factor:0.25')
clip.getvideo ()

In [ ]:
t = video.framerate
rate = int (t [0]) / int (t [1])

create_grid_video()

In [ ]:
rate

In [ ]:
# create video grid: list of [video file, incident-offset, (left,top,right,bottom)]
# descs.append ((vid.filename, incident-offset, (left, top, right, bottom)))
#create_grid_video(descs,duration,speed=0.5) # duration is in seconds!!